## Analyzing Hamlet

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import src.data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from collections import OrderedDict

from IPython.display import display

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)

In [ ]:
import itertools

In [ ]:
file_names = {
    'aurora':'2020-01-01-till-2021-02-24-aurora.csv',
    'hamlet':'2020-01-01-till-2021-02-24-hamlet.csv',
    'mercandia':'2020-01-01-till-2021-02-24-mercandia-iv.csv',
    'tycho-brahe':'2020-01-01-till-2021-02-24-tycho-brahe.csv',
}

In [ ]:
dfs = OrderedDict()

for ship_name, file_name in file_names.items():
    
    file_path = os.path.join(src.data.path_ships,file_name)
    reader = pd.read_csv(file_path, chunksize=1000, iterator=True)  # Loading a small part of the data
    dfs[ship_name] = next(reader)
    

In [ ]:
for ship_name, df in dfs.items():
    
    display(df.describe())
    

In [ ]:
file_path = os.path.join(src.data.path_ships,file_names['aurora']) 
reader = pd.read_csv(file_path, chunksize=1000000, iterator=True)  # Loading a small part of the data
df_raw = next(reader)
df_raw.set_index('Tidpunkt [UTC]', inplace=True)
df_raw.index = pd.to_datetime(df_raw.index)

In [ ]:
mask = df_raw['Fart över grund (kts)']>1
df = df_raw.loc[mask].copy()

In [ ]:
df.hist(column='Kurs över grund (deg)', bins=1000)

In [ ]:
mask = df_raw['Kurs över grund (deg)'] < 150
df_direction_1 = df.loc[mask]
df_direction_1.describe()

In [ ]:
df_direction_1.plot(x='Longitud (deg)', y = 'Latitud (deg)', style='.', alpha=0.005)

In [ ]:
deltas = []
for i in range(1,5):
    sin_key = 'Sin EM%i ()' % i
    cos_key = 'Cos EM%i ()' % i
    delta_key = 'delta_%i' % i
    deltas.append(delta_key)

    df_direction_1[delta_key] = np.arctan2(df_direction_1[sin_key],df_direction_1[cos_key])

df_plot = df_direction_1.loc['2020-01-01 01:00':'2020-01-01 02:00']
    
df_plot.plot(y=['Kurs över grund (deg)','Stävad kurs (deg)'],style='.')
df_plot.plot(y='Fart över grund (kts)',style='.')

df_plot.plot(y=deltas,style='.')



In [ ]:
df_direction_1.head()

In [ ]:
(df_direction_1['Sin EM1 ()']**2 +   df_direction_1['Cos EM1 ()']**2).hist()

In [ ]:
df_direction_1.columns

In [ ]:

descriptions = pd.Series(index = df_direction_1.columns.copy())
descriptions['Latitud (deg)'] = 'Latitud (deg) (WGS84?)'
descriptions['Longitud (deg)'] = 'Longitud (deg) (WGS84?)'
descriptions['Effekt DG Total (kW)'] = '?'
descriptions['Effekt EM Thruster Total (kW)'] = ''
descriptions['Sin EM1 ()'] = ''
descriptions['Sin EM2 ()'] = ''
descriptions['Sin EM3 ()'] = ''
descriptions['Sin EM4 ()'] = ''
descriptions['Cos EM1 ()'] = ''
descriptions['Cos EM2 ()'] = ''
descriptions['Cos EM3 ()'] = ''
descriptions['Cos EM4 ()'] = ''
descriptions['Fart över grund (kts)'] = 'GPS fart'
descriptions['Stävad kurs (deg)'] = 'Kompas kurs'
descriptions['Kurs över grund (deg)'] = 'GPS kurs'
descriptions['Effekt hotell Total (kW)'] = ''
descriptions['Effekt Consumption Total (kW)'] = ''
descriptions['Förbrukning GEN alla (kg/h)'] = '?'
descriptions['delta_1'] = 'Thruster angle 1'
descriptions['delta_2'] = 'Thruster angle 2'
descriptions['delta_3'] = 'Thruster angle 3'
descriptions['delta_4'] = 'Thruster angle 4'

df_numenclature = pd.DataFrame(descriptions, columns=['Description'])
df_numenclature